# Infinitesimal Rigidity

In this notebook, we are going to be exploring the conept of *infinitesimal rigidity*. In particular, We will be looking at

* Trivial and non-trivial flexes
* The Rigidity Matrix

## Imports and helpful functions

Before we begin, we first import packages and define some functions that we will use throughout.


In [1]:
from sympy import Matrix, solve, symbols

def Vector(*args):
    return Matrix([[x] for x in args])

## Trivial and non-trivial flexes

Recall that, for a framework $(G,p)$ in dimension $d$, a vector configuration $v$ of $n$ vectors in $\mathbb{R}^d$ is an **infinitesimal flex** of $(G,p)$ if
$$
    \newcommand\iprod[2]{\left\langle {#1},{#2}\right\rangle}
    \iprod{p_j - p_i}{v_j - v_i} = 0
    \qquad \text{(all edges $ij\in E$)}.
$$
An infinitesimal flex is **trivial** if the stronger condition
$$
    \newcommand\iprod[2]{\left\langle {#1},{#2}\right\rangle}
    \iprod{p_j - p_i}{v_j - v_i} = 0
    \qquad \text{(all vertices $i\neq j\in V$)}
$$
holds.

As the name might sugest, all frameworks permit trivial motions. Therefore, we are usually interested in the non-trivial case, as these flexes correspond to non-euclidean motions in the plane. Below we create a function that finds a basis for the space of these non-trivial motions, if such a space exists.


In [2]:
def infFlex(config, neighbors):
    """
    Calculates a non-trivial infinitesiaml flex inf such a flex exists.

    Parameters
    ----------
    config : list
        List of position vectors. Number of points must be greater than the
        dimension of the space.
    neighbors : dict
        Dictionary that stores the indices of neighboring vertices.

    Returns
    -------
    flex : list
        A non-trivial infinitesimal flex if one exists, otherwise all zeros.

    """

    dim = len(config[0])
    edges = [config[i]-config[j] for i in neighbors for j in neighbors[i] if i<j]

    # Create a list of symbols that will represent our flex
    flex = []
    for i, p in enumerate(config):
        flex.append(Vector(*[symbols('x' + str(i) + str(k)) for k in range(dim)]))

    # Create a list of constraints that fix the first point in the
    # configuration, then constrains the second flex to be in the line between
    # p0 and p1, the third flex to lie in the plane defined by p0, p1 and p2
    # and so on. This is possible if n > dim and the configuration is in
    # general position. We do this by finding the nullspace of the matrix of
    # vectors that define the hyperplane, which gives a normal vector. To
    # ensure the flex remains in the hyperplane, we enforce that the dot
    # product between the normal and the flex is zero.
    nonTrivConstraints = []
    for entry in flex[0]:
        nonTrivConstraints.append(entry)

    p0 = config[0]
    for i in range(1,dim):
        vects = [p0 - p for p in config[1:i+1]]
        fixedHypPlane = Matrix([v.transpose() for v in vects])
        normal = fixedHypPlane.nullspace()[0]

        constraint = flex[i].dot(normal)
        nonTrivConstraints.append(constraint)

    # Solve the system and apply the constraints to flex
    nonTrivSoln = solve(nonTrivConstraints)
    flex = [delta.subs(nonTrivSoln) for delta in flex]

    # Create the constraints necessary for infinitesimal rigidity
    flexEqns = [flex[i]-flex[j] for i in neighbors for j in neighbors[i] if i<j]
    rigidConstraints = [edge.dot(flexEqns[i]) for i,edge in enumerate(edges)]

    # Solve the system
    soln = solve(rigidConstraints, dict=True)
    flex = [delta.subs(soln[0]) for delta in flex]

    return flex

### Examlpes

To see how we can use this function, we look at the following examples:

* Octahedron with a flattened vertex
* Triangle
* Square
* Square with a brace
* Two adjoined squares with braces
* Two adjoined squares with braces but in $\mathbb{R}^3$


In [3]:
# Flattened Octahedron
a1 = Vector(-2,0,0)
a2 = Vector(4,2,0)
a3 = Vector(2,0,0)
a4 = Vector(4,-2,0)
a5 = Vector(1,0,3)
a6 = Vector(1,0,-3)

verts = [a1,a2,a3,a4,a5,a6]
nbs = {0:[1,3,4,5], 1:[0,2,4,5], 2:[1,3,4,5], 3:[0,2,4,5], 4:[0,1,2,3], 5:[0,1,2,3]}
a = infFlex(verts, nbs)

In [4]:
# Triangle
b1 = Vector(0,0)
b2 = Vector(1,0)
b3 = Vector(0,1)

verts = [b1,b2,b3]
nbs = {0:[1,2], 1:[0,2], 2:[0,1]}
b = infFlex(verts, nbs)

In [5]:
# Square
c1 = Vector(0,0)
c2 = Vector(1,0)
c3 = Vector(0,1)
c4 = Vector(1,1)

verts = [c1,c2,c3,c4]
nbs = {0:[1,2], 1:[0,3], 2:[0,3], 3:[1,2]}
c = infFlex(verts, nbs)

In [6]:
# Square with brace
d1 = Vector(0,0)
d2 = Vector(1,0)
d3 = Vector(0,1)
d4 = Vector(1,1)

verts = [d1,d2,d3,d4]
nbs = {0:[1,2,3], 1:[0,3], 2:[0,3], 3:[0,1,2]}
d = infFlex(verts, nbs)

In [7]:
# Two joined squares with braces
e1 = Vector(0,0)
e2 = Vector(1,0)
e3 = Vector(2,0)
e4 = Vector(0,1)
e5 = Vector(1,1)
e6 = Vector(2,1)

verts = [e1,e2,e3,e4,e5,e6]
nbs = {0:[1,3,4], 1:[0,2,4,5], 2:[1,5], 3:[0,4], 4:[0,1,3,5], 5:[1,2,4]}
e = infFlex(verts, nbs)

In [8]:
# Two joined squares with braces, but in R^3
f1 = Vector(0,0,0)
f2 = Vector(1,0,0)
f3 = Vector(2,0,0)
f4 = Vector(0,1,0)
f5 = Vector(1,1,0)
f6 = Vector(2,1,0)

verts = [f1,f2,f3,f4,f5,f6]
nbs = {0:[1,3,4], 1:[0,2,4,5], 2:[1,5], 3:[0,4], 4:[0,1,3,5], 5:[1,2,4]}
f = infFlex(verts, nbs)